# Part 1: GRASS GIS intro

In this first part we will demonstrate starting GRASS GIS, creating new project and basic data visualization.
We will first use the GUI and then we will replicate it in the notebook environment running on mybinder.com.

## GUI intro

## GRASS GIS in a notebook

### Notebook environment

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

However we will also show how to use GRASS from shell, for that we will use IPython magic:

In [ ]:
%%bash
pwd

### Start GRASS GIS

Create new empty location (project) that uses projection [UTM zone 17 N](https://epsg.io/6346).

![Create new location](img/new_location.gif)

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created location. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init(os.path.expanduser("~/grassdata"), "dix_park", "PERMANENT")

### Import data

Import prepared DSM, DTM and ortho maps. If you run this workshop locally (not in Binder), download the data [here](http://fatra.cnr.ncsu.edu/foss4g2021/). In this case no reprojection is done since the data CRS matches the location's CRS.

![Import raster data](img/import_raster.gif)

We also set the computational region based on the imported raster (more on that later).

In [ ]:
%%bash
r.import input=dsm.tif output=dsm resample=bilinear
r.import input=ground.tif output=ground
r.import input=ortho.tif output=ortho

# set the region to match ortho raster
g.region raster=ortho

Download OSM data using Overpass Turbo.

We extract roads for our study area. The query should run upon running the cell below, so
it should be enough to *press Export - download as GeoJSON* and save it as `roads.geojson`.
This will download the file on your local machine, so if you are running this in Jupyter Lab on Binder, upload it there with *Upload files* in the top left corner.

In [ ]:
import IPython
IPython.display.IFrame('https://overpass-turbo.eu/s/1aGu', width=800, height=500)

Now we import the downloaded file. This time the data will be automatically reprojected because the CRS doesn't match.

![Import vector data](img/import_vector.gif)

In [ ]:
%%bash
v.import input=roads.geojson output=roads

Let's look at the available data in our location:

In [ ]:
%%bash
g.list type=raster,vector -m -t

### Data visualization

![Display raster and vector data, set color](img/display.gif)

We will call d.rast/d.vect modules using the new GRASS Jupyter API:

In [ ]:
# Start a GrassRenderer
img = gj.GrassRenderer()
# Add a raster and vector to the map
img.d_rast(map="ground")
img.d_vect(map="roads")
img.d_legend(raster="ground", flags='b')
# Display map
img.show()

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ortho")
img.d_vect(map="roads", width="2", color="yellow", where="name = 'Umstead Drive'")
img.show()

Here is how we can visualize data with folium:

In [ ]:
fig = gj.InteractiveMap(width=600)
# Add vector and layer control to map
fig.add_vector("roads")
fig.show()

We can also visualize data in 3D. Here we drape the ortho over the DSM.

![3D visualization](img/3D_visualization.gif)

In [ ]:
img = gj.Grass3dRenderer()
img.render(elevation_map="dsm", color_map="ortho",
           position=(0.5, 1), height=3000, perspective=12)
img.show()

In [ ]:
img = gj.Grass3dRenderer()
img.render(elevation_map="dsm", resolution_fine=1, color_map="ortho",
           light_position=(1, 0, 0.5),
           position=(0.75, 0.35), height=1500, perspective=10)
img.show()

## GRASS GIS modules

GRASS functionality is available through modules (tools, functions). There are over 500 different modules in the core distribution and over 300 addon modules that can be used to prepare and analyze data.

Modules respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some modules have multiple dots in their names. For example, modules staring with v.net.* deal with vector network analysis and r.in.* modules import raster data into GRASS GIS spatial database.


### Finding and running a module

![Find modules](img/search_module.gif)

To find a module for your analysis, type the term into the **search box within the Modules tab** in the Layer Manager or just browse the module tree.
Most modules are also available from the **main menu**. For example, to find information about a raster map, use: *Raster → Reports and statistics → Basic raster metadata*.

If you already know the name of the module, you can just use it in the command line. The GUI offers a **Console tab with command line** specifically built for running GRASS GIS modules. If you type the module name there, you will get suggestions for automatic completion of the name. After pressing Enter, you will get the GUI dialog for the module.

Last but not least, there is a module for finding modules:

In [ ]:
%%bash
g.search.modules keyword=zonal

In [ ]:
%%bash
r.univar --help

### GUI  vs command line vs Python

GRASS modules can be executed either through the GUI, command line or Python interfaces. The GUI offers a user-friendly approach to execute modules where the user can navigate to data layers that they would like to analyze and modify processing options with simple check boxes.
The GUI also offers an easily accessible manual on how to execute a model. The command line interface allows users to execute a module using command prompts specific to that module. This is handy when you are running similar analyses with minor modification or are familiar with the module commands for quick efficient processing.

In this example we will show how the same module can be executed with the GUI, in shell and in Python:

![v.extract](img/v.extract.png) &nbsp;&nbsp;&nbsp;&nbsp;
![v.extract](img/v.extract.gif)

In [ ]:
%%bash
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segments

In [ ]:
gs.run_command("v.extract", input="roads", where="name = 'Umstead Drive'", output="umstead_drive_segments")

### Computational region

Computational region is an important raster concept in GRASS GIS.
Before we use a module to compute a new raster map, we must properly set the computational region. All raster computations will be performed in the specified extent and with the given resolution.
Among other things, this allows us to easily subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by region extent and raster resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets
 * advantages: keeps your results consistent, avoids clipping, for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis
 * run `g.region -p` or in menu *Settings - Region - Display region* to see current region settings
 
 ![Computational region](img/region.gif)

In [ ]:
%%bash
g.region -p

The most common way to set region is **based on a raster map** - both extent and resolution. If the raster is added to Layer Manager, right click on it and select *Set computational region from selected map*. Or run the g.region module (we include -p flag to always see the resulting region):

In [ ]:
%%bash
g.region -p raster=dsm

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector maps do not have any resolution - at least not in the way raster maps do). In the GUI, this can be done in the same way as for the raster map. In the command line, it looks like this:


In [ ]:
%%bash
g.region -p vector=roads | tail

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number. If that's not desired, we can set it explicitly using -a flag and parameter res. Now the resolution is aligned to even multiples of 2 (the units are the units of the current location, in our case meters):


In [ ]:
%%bash
g.region -p vector=roads res=2 -a

Often we need to set the computational extent based on a vector map, but take the resolution and alignment from a raster map:

In [ ]:
%%bash
g.region -p vector=roads align=dsm

## Python API

There are two Python APIs for accessing a module's functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass78/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows. Here we will be using Python Scripting Library (`import grass.script as gs`)
as it is simple and straightforward to use.
 

GRASS GIS Python Scripting Library provides functions to call GRASS modules within scripts as subprocesses. The most often used functions include:

 * [script.core.run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [script.core.read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [script.core.parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [script.core.write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file


It also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [script.raster.raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [script.vector.vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [script.core.list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [script.core.region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`

**Note for GUI:** The simplest way to execute the Python code which uses GRASS GIS packages is to use the Simple Python editor integrated in GRASS GIS (accessible from the toolbar or the Python tab in the Layer Manager). Another option is to write the Python code in your favorite plain text editor like Notepad++ (note that Python editors are plain text editors). Then run the script in GRASS GIS using the main menu *File -> Launch script*.

![Run Python in GUI](img/python.gif)

Let's use Python to compute viewshed:

In [ ]:
# 1. set computational region based on DSM
gs.run_command("g.region", raster="dsm")
# 2. Compute viewshed, flag 'b' is for binary (0 and 1) output 
gs.run_command("r.viewshed", input="dsm", output="viewshed", flags="b", coordinates=(711260, 3960860))
# 3. Compute basic univariate statistics, flag 'g' is for parsable output
univar = gs.parse_command("r.univar", map="viewshed", flags='g')
# 4. Get current region settings to get cell size
region = gs.region()
cell_size = region["nsres"] * region["ewres"]
percentage = 100 * float(univar['sum']) / float(univar['n'])
area = cell_size * float(univar['sum'])
print(f"Percentage of visible area is {percentage:.2f}%, which is {area} ha")

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ortho")
# select only cells with value 1 to visualize
img.d_rast(map="viewshed", values=1)
img.show()